In [ ]:
import ipyslurm

Pass login arguments to the constructor.

In [ ]:
slurm = ipyslurm.Slurm('my.server', username='my.username')

An example bash script that dynamically creates a job array.

In [ ]:
script = r"""
job_name=ipyslurm
jobs=("A" "B" "C")

if [[ -z "$SLURM_JOB_ID" ]]; then
    args=(
        "--array 1-${#jobs[@]}"
        "--job-name \"$job_name\""
        "--mem 1GB"
        "--output $HOME/ipyslurm.%a.log"
        "--time 00:10:00")
    echo ${args[@]}
    exit
fi

job_id=$(($SLURM_ARRAY_TASK_ID - 1))
echo ${jobs[$job_id]}
"""
stdout = slurm.command(script)
job = slurm.sbatch(script, args=stdout)
slurm.tail(job)

An example Python script that dynamically creates a job array.

In [ ]:
script = r"""
#!/usr/bin/env python
import itertools
import os

job_name = 'ipyslurm'
parameter1s = 'A', 'B', 'C'
parameter2s = 1, 2, 3
jobs = tuple(itertools.product(parameter1s, parameter2s))

if os.getenv('SLURM_JOB_ID') is None:
    print(' '.join((
        f'--array 1-{len(jobs)}',
        f'--job-name "{job_name}"',
        f'--output $HOME/ipyslurm.%a.log',
    )))
    raise SystemExit

job_id = int(os.getenv('SLURM_ARRAY_TASK_ID')) - 1
parameter1, parameter2 = jobs[job_id]
print(parameter1, parameter2)
"""
stdout = slurm.command(script)
job = slurm.sbatch(script, args=stdout)
slurm.tail(job)

In [ ]:
print(slurm.command('rm ipyslurm.*.log'))

Create empty files to demonstrate `sftp` below.

In [ ]:
print(slurm.command(r"""
mkdir -p $HOME/ipyslurm_example
for i in `seq 1 9`; do
    touch $HOME/ipyslurm_example/$i
done
"""))

Pass `-r` to apply an operation recursively and `-a` to resume a partial transfer.  
See interactive commands section of https://man.openbsd.org/sftp for available commands.

In [ ]:
slurm.sftp(r"""
cd $HOME
lcd %USERPROFILE%
get -ra ipyslurm_example
put -ra ipyslurm_example
rm -r ipyslurm_example
lrm -r ipyslurm_example
""")

Pass slurm instance to `ipyslurm` magics.

In [ ]:
%load_ext ipyslurm

In [ ]:
%%scommand --instance slurm
echo "$HOME"

Descruction ensures logout.

In [ ]:
del slurm